In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier

# Définir les colonnes catégoriques, binaires et numériques
cat_cols = [
    "NAME_CONTRACT_TYPE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE",
    "NAME_FAMILY_STATUS_Civil", "EMERGENCYSTATE_MODE", "ORGANIZATION_TYPE",
    "NAME_EDUCATION_TYPE", "HOUSETYPE_MODE"
]

binary_cols = ["CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "FLAG_WORK_PHONE"]

num_cols = [
    "EXT_SOURCE_2", "INSTAL_AMT_PAYMENT_MIN", "EXT_SOURCE_3",
    "INSTAL_PAYMENT_DIFF_MAX", "PREV_NAME_YIELD_GROUP_high_MEAN",
    "APPROVED_AMT_DOWN_PAYMENT_MIN", "OWN_CAR_AGE", "BURO_AMT_ANNUITY_MEAN",
    "PREV_PRODUCT_COMBINATION_Card_Street_MEAN", "CNT_CHILDREN",
    "PREV_PRODUCT_COMBINATION_Cash_X_Sell__high_MEAN", "LIVE_CITY_NOT_WORK_CITY"
]

# Transformer les colonnes binaires
binary_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(drop=None, handle_unknown="ignore"))  # Inclure NaN comme catégorie
])

# Transformer les colonnes numériques
num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),  # Remplacer les NaN par la médiane
    ("scaler", StandardScaler())  # Standardiser les valeurs
])

# Transformer les colonnes catégoriques
cat_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # Colonnes dummy avec NaN
])

# Combiner les transformations
preprocessor = ColumnTransformer(transformers=[
    ("num", num_transformer, num_cols),       # Colonnes numériques
    ("cat", cat_transformer, cat_cols),      # Colonnes catégoriques
    ("binary", binary_transformer, binary_cols)  # Colonnes binaires
])

# Définir SMOTE
smote = SMOTE(random_state=42)

# Définir le modèle avec des paramètres choisis
lgbm_classifier = LGBMClassifier(
    learning_rate=0.8,
    n_estimators=200,
    max_depth=10,
    min_child_samples=100,
    subsample=0.7,
    random_state=76
)

# Construire le pipeline avec SMOTE et le modèle
pipeline = ImbPipeline(steps=[
    ("preprocessor", preprocessor),     # Étape de prétraitement
    ("smote", smote),                   # Étape de rééchantillonnage
    ("classifier", lgbm_classifier)    # Modèle
])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

# Entraîner le pipeline sur les données d'entraînement
pipeline.fit(X_train, Y_train)

# Faire des prédictions sur les données de test
y_pred = pipeline.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy : {accuracy:.4f}")

# Sauvegarder le pipeline dans un fichier
joblib.dump(pipeline, '../models/preprocessing_pipeline_with_model.joblib')


NameError: name 'X' is not defined